In [1]:
import requests
import codecs
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import ast
import urllib2

In [2]:
def parse_one_page(page_url):
    
    page = requests.get(page_url).text
    text = BeautifulSoup(page, "html.parser")
    
    result_lst = text.find_all('li', { 'class': 'hover propertyCard property-data-elem clickable' })
    result = {}

    for i, apt in enumerate(result_lst):
        keyid = result_lst[i].find('div', {'class': "man typeTruncate"}).get_text().encode('utf-8').strip()    
        result[keyid] = []
        dict_address= {}
        dict_Coordinates = {}

        for room in apt.find_all('li', {'class': 'pvs mvn pll'}):
            try:
                room_info = room.div.get('data-floorplan').encode('utf8')
                room_info2 = eval(room_info.replace('null', 'None'))
                final_room_info = {key: room_info2[key] for key in room_info2 if key in  
                         [  'numBedrooms',   'planPriceFrom' ,  'planMinimumSquareFeet' ]}
            except AttributeError:
                pass
            else:
                result[keyid].append(final_room_info)    
        
    return result


In [3]:
# url = "http://www.trulia.com/for_rent/New_York,NY/1_p"

def Apt_Type(url):
    result = parse_one_page(url)
    List1 =[]
    for apts in result:
        dictTable1 = {'T_studioArea':None , 'T_studioPrice':None , 
                     'T_1bedRoomArea':None , 'T_1bedRoomPrice':None ,
                     'T_2bedRoomArea':None ,'T_2bedRoomPrice':None  }
        for i,dictList in enumerate(result[apts]):
            dictbox = dictList
            dictTable1['AptName'] = apts

            if dictbox['numBedrooms'] == 0:
                dictTable1['T_studioArea'] = dictbox['planMinimumSquareFeet']
                dictTable1['T_studioPrice'] = dictbox['planPriceFrom']

            if dictbox['numBedrooms'] == 1:
                dictTable1['T_1bedRoomArea'] = dictbox['planMinimumSquareFeet']
                dictTable1['T_1bedRoomPrice'] = dictbox['planPriceFrom']   

            if dictbox['numBedrooms'] == 2:
                dictTable1['T_2bedRoomArea'] = dictbox['planMinimumSquareFeet']
                dictTable1['T_2bedRoomPrice'] = dictbox['planPriceFrom']
        List1.append(dictTable1)
    return List1

    

In [4]:
def Apt_Location(page_url):    
    page = requests.get(page_url).text
    text = BeautifulSoup(page, "html.parser")
    result_lst = text.find_all('li', { 'class': 'hover propertyCard property-data-elem clickable' })

    # result = {}
    list2 = []
    for i, apt in enumerate(result_lst):        
        dictTable2= {}
        dictTable2['AptName'] = result_lst[i].find('div', {'class': "man typeTruncate"}).get_text().encode('utf-8').strip() 
        dictTable2['Street'] = apt.find('h2', {'itemprop': "streetAddress"}).get_text().encode('utf-8').strip()
        dictTable2['City'] = apt.find('span', {'itemprop': "addressLocality"}).get_text().encode('utf-8').strip()
        dictTable2['State'] = apt.find('span', {'itemprop': "addressRegion"}).get_text().encode('utf-8').strip()
        dictTable2['Postal'] = apt.find('span', {'itemprop': "postalCode"}).get_text().encode('utf-8').strip()

        LatitudeTags  = apt.find_all('meta', {'itemprop': 'latitude'})
        LongitudeTags  = apt.find_all('meta', {'itemprop': 'longitude'})
        dictTable2["latitude"] = [tag.get('content').encode('utf-8') for tag in LatitudeTags][0]
        dictTable2["longitude"] = [tag.get('content').encode('utf-8') for tag in LongitudeTags][0]
        list2.append(dictTable2)
    return list2

In [5]:
def Apt_AllInfo(url):
    list_AptLoc = Apt_Location(url)
    list_AptType = Apt_Type(url)
    df_AptLoc = pd.DataFrame(list_AptLoc, columns= ["AptName", "City","Postal","State","Street","latitude","longitude"])
    df_AptType= pd.DataFrame(list_AptType, columns= ["AptName", "T_1bedRoomArea","T_1bedRoomPrice","T_2bedRoomArea",
                                                   "T_2bedRoomPrice", "T_studioArea","T_studioPrice"])
    df_AllApt_Info = pd.merge(left=df_AptLoc,right=df_AptType, left_on='AptName', right_on='AptName')
    return df_AllApt_Info
# AptName	T_1bedRoomArea	T_1bedRoomPrice	T_2bedRoomArea	T_2bedRoomPrice	T_studioArea	T_studioPrice    

In [6]:
def SchoolCrimeInfo(page_url):
    page = requests.get(page_url).text
    text = BeautifulSoup(page, "html.parser")
    result_lst = text.find_all('li', { 'class': 'hover propertyCard property-data-elem clickable' })

    list3 = []
    for i, apt in enumerate(result_lst):
        dictTable2 = {'ElementarySchool_rating':None,'MiddleSchool_rating':None,'HighSchool_rating':None, "CrimeLevel": None}
        dictTable2['AptName'] = result_lst[i].find('div', {'class': "man typeTruncate"}).get_text().encode('utf-8').strip()

        anchors = result_lst[i].find('a', {'class': "primaryLink pdpLink activeLink"})['href']
        site = "http://www.trulia.com/" + anchors
        response = requests.get(site).text
        text = BeautifulSoup(response, "html.parser")    
        result_lst3 = text.find_all('div', { 'id': 'schoolsAtAGlance' })
        for result in result_lst3:        
            body1 = result.find_all('div', { 'class': 'boxBody mvs' })[0].get_text().encode('utf-8').strip()      
            body1 = filter(bool, map(lambda x: x.strip(), body1.split('\n')))

            for i in range(len(body1)):
                if (body1[i] == 'Elementary'):
                     dictTable2['ElementarySchool_rating']= body1[i + 1]
                if (body1[i] == 'Middle'):
                     dictTable2['MiddleSchool_rating']= body1[i + 1]
                if (body1[i] == 'High'):
                     dictTable2['HighSchool_rating']= body1[i + 1] 
        result_lst4 = text.find_all('div', { 'data-action': 'showCrimePopup' })    
        for result in result_lst4:
    #         Crime = {"CrimeLevel": None}
            body2 = result.find('div', { 'class': 'boxBody media pvs h5' }).get_text().encode('utf-8').strip()
            body2 = filter(bool, map(lambda x: x.strip(), body2.split('\n')))      
            dictTable2["CrimeLevel"] = body2[0]
    #         list3.append(Crime)

        list3.append(dictTable2)
    df_School_Crime = pd.DataFrame(list3)
    return df_School_Crime



In [10]:
import time
DF_all = pd.DataFrame()
for i in range(1,4):
    urls = "http://www.trulia.com/for_rent/New_York,NY/" + str(i) + "_p"
    print urls
    Df_Temp = Apt_AllInfo(urls)
    DF_all = pd.concat([DF_all, Df_Temp])
    time.sleep(1)
DF_all
DF_all.to_excel("trulia_1_AptInfo.xls")


http://www.trulia.com/for_rent/New_York,NY/1_p
http://www.trulia.com/for_rent/New_York,NY/2_p
http://www.trulia.com/for_rent/New_York,NY/3_p


In [11]:
# school Crime Takes lot of time
DF_SchoolCrime_All = pd.DataFrame()
for i in range(1,4):
    urls = "http://www.trulia.com/for_rent/New_York,NY/" + str(i) + "_p"
    Df_Temp = SchoolCrimeInfo(urls)
    DF_SchoolCrime_All = pd.concat([DF_SchoolCrime_All, Df_Temp])
    time.sleep(1)
DF_SchoolCrime_All
DF_SchoolCrime_All.to_excel("trulia_1_SchoolCrimes.xls")